### Sections

- [Loading patientLabelNPI.csv](#Loading-patientLabelNPI.csv)
- [Loading diabetes_all_claims](#Loading-diabetes_all_claims)
- [Merging diabetes_all_claim and patient_Label_npi](#Merging-diabetes_all_claim-and-patient_Label_npi)
- [Loading CCS_Codes Description](#Loading-CCS_Codes-Description)
- [Making summary of CCS_codes for 9K diabetic patients](#Making-summary-of-CCS_codes-for-9K-diabetic-patients)
- [Clustering](#Performing-Clustering-Using-Cosine-Similarity)
- [Performance evaluation measures on finalised clusters](#Performance-evaluation-measures-on-finalised-clusters)
- [Grouping to see patients in each comm-cluster combination](#Grouping-to-see-patients-in-each-comm-cluster-combination)
- [Adding Gender ,Pcal,Average Age information to community cluster](#Adding-Gender-,-Pcal-,-Average-Age-information-to-community-cluster)
- [Defining NPI Communities](#Defining-NPI-Communities)
- [Adding PQI measurement euid-npi-cluster](#Adding-PQI-measurement-euid-npi-cluster)

# Loading patientLabelNPI.csv

PatientLabelNPI has the mapping new_euid - comm_npi

In [2]:
import pandas as pd  
import numpy as np 
import math

In [3]:
cols=['new_euid','comm_npi']
type ={'new_euid':'str','comm_npi':'int64'}
patient_label_npi=pd.read_csv("../../Data/Diabetic_Patient_Clustering/patientLabelNPI.csv",usecols=cols,dtype=type)

In [4]:
patient_label_npi.head()

,new_euid,comm_npi
0,44470,1
1,498737,6
2,33055,1
3,187443,6
4,118528,1


In [5]:
patient_label_npi.groupby(['comm_npi']).new_euid.nunique()

comm_npi
1    2947
2    1882
3    2144
4     386
5     842
6     754
Name: new_euid, dtype: int64

In [6]:
patient_label_npi.shape 

(8955, 2)

In [7]:
#unique patients in patient_Label_npi
len(patient_label_npi['new_euid'].unique()) 

8955

In [8]:
#checking distinct euid length
patient_label_npi.groupby(patient_label_npi['new_euid'].str.len()).agg({'new_euid':np.size}) 

,new_euid
new_euid,
2,2
3,20
4,338
5,1192
6,7403


In [9]:
#checking if a patient (euid) is in more than one community
tmp=pd.DataFrame(patient_label_npi.groupby(['new_euid']).comm_npi.nunique())
tmp[tmp['comm_npi']>1].shape  
#No such patients 

(0, 1)

# Loading diabetes_all_claims

diabetes_all_claims has mapping new_euid - ccs codes

In [10]:
cols=['new_euid','cli_dx1_ccs','cli_dx2_ccs','cli_dx3_ccs','cli_dx4_ccs'] 
type={'new_euid':'str','cli_dx1_ccs':'str','cli_dx2_ccs':'str','cli_dx3_ccs':'str','cli_dx4_ccs':'str'}
diabetes_all_claims=pd.read_csv("../../Data/Diabetic_Patient_Clustering/diabetes_all_claims.csv",usecols=cols,dtype=type)

In [11]:
diabetes_all_claims.head() 

,new_euid,cli_dx1_ccs,cli_dx2_ccs,cli_dx3_ccs,cli_dx4_ccs
0,0604865,NaN,NaN,NaN,NaN
1,0604865,NaN,NaN,NaN,NaN
2,0604865,NaN,NaN,NaN,NaN
3,0604865,NaN,NaN,NaN,NaN
4,0604865,NaN,NaN,NaN,NaN


In [12]:
#Total Observation
diabetes_all_claims.shape

(790086, 5)

In [13]:
#Total unique patients
len(diabetes_all_claims['new_euid'].unique()) 

9843

In [14]:
#null observation
diabetes_all_claims.isnull().sum() 

new_euid            0
cli_dx1_ccs    292966
cli_dx2_ccs    617053
cli_dx3_ccs    684626
cli_dx4_ccs    729090
dtype: int64

In [15]:
#checking size of new_euid length
diabetes_all_claims.groupby(diabetes_all_claims['new_euid'].str.len()).agg({'new_euid':np.size}) 

,new_euid
new_euid,
7,790086


In [16]:
#stripping front 0
diabetes_all_claims['new_euid']=diabetes_all_claims['new_euid'].map(lambda x:str(x).lstrip('0'))

In [17]:
#checking size of new_euid length after stripping front 0
diabetes_all_claims.groupby(diabetes_all_claims['new_euid'].str.len()).agg({'new_euid':np.size}) 

,new_euid
new_euid,
2,187
3,1649
4,32212
5,106727
6,649311


# Merging diabetes_all_claim and patient_Label_npi

merging both of these files to get the mapping new_euid-comm_npi-ccs codes

In [18]:
diabetic_patients_comm_ccs=pd.merge(patient_label_npi,diabetes_all_claims) 

In [19]:
diabetic_patients_comm_ccs.head() 

,new_euid,comm_npi,cli_dx1_ccs,cli_dx2_ccs,cli_dx3_ccs,cli_dx4_ccs
0,44470,1,NaN,NaN,NaN,NaN
1,44470,1,NaN,NaN,NaN,NaN
2,44470,1,NaN,NaN,NaN,NaN
3,44470,1,NaN,NaN,NaN,NaN
4,44470,1,NaN,NaN,NaN,NaN


In [20]:
#unique diabetic patients after merging
len(diabetic_patients_comm_ccs['new_euid'].unique()) 

8955

In [21]:
#check on null
diabetic_patients_comm_ccs.isnull().sum() 

new_euid            0
comm_npi            0
cli_dx1_ccs    268736
cli_dx2_ccs    569704
cli_dx3_ccs    632085
cli_dx4_ccs    673294
dtype: int64

In [39]:
#checking unique values within ccs_code
# df=diabetic_patients_comm_ccs
# cols=df.columns.tolist()
# cols=cols[2:]
# for col in cols:
#     print(diabetic_patients_comm_ccs[col].unique()) ; 

In [22]:
#filling NA's as 0
diabetic_patients_comm_ccs=diabetic_patients_comm_ccs.fillna(0)

In [23]:
#checking null count after filling NA's as 0
diabetic_patients_comm_ccs.isnull().sum()

new_euid       0
comm_npi       0
cli_dx1_ccs    0
cli_dx2_ccs    0
cli_dx3_ccs    0
cli_dx4_ccs    0
dtype: int64

In [24]:
#checking head
diabetic_patients_comm_ccs.head() 

,new_euid,comm_npi,cli_dx1_ccs,cli_dx2_ccs,cli_dx3_ccs,cli_dx4_ccs
0,44470,1,0,0,0,0
1,44470,1,0,0,0,0
2,44470,1,0,0,0,0
3,44470,1,0,0,0,0
4,44470,1,0,0,0,0


In [25]:
del diabetic_patients_comm_ccs['comm_npi']

# Loading CCS_Codes Description

In [26]:
ccs_codes=pd.read_csv('../../Data/ccs_code.csv')  #reading ccs code

#Adding 'CCS' ahead of CCS code number
ccs_codes['ccs_code']= "CCS"+ ccs_codes.START.map(str) #Adding CCS Codes 


In [27]:
ccs_codes.head() 

,START,LABEL,ccs_code
0,1,Tuberculosis,CCS1
1,10,Immuniz/scrn,CCS10
2,100,Acute MI,CCS100
3,101,Coron athero,CCS101
4,102,Chest pain,CCS102


Making Binary Matrix with new-euid and each of 283 CCS codes as column

In [28]:
cols=list(ccs_codes['ccs_code'])
cols.insert(0,'new_euid') 
diabetic_patients_binary=pd.DataFrame(columns=cols,index=diabetic_patients_comm_ccs.index) 
diabetic_patients_binary.head()  

,new_euid,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
##Code to fill up binary Matrix as per each patients ccs_codes 
for index,row in diabetic_patients_comm_ccs.iterrows(): 
    if(row['cli_dx1_ccs'] !=0): 
        col="CCS" +str(row['cli_dx1_ccs'])
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid']
    if(row['cli_dx2_ccs'] !=0): 
        col="CCS" +str(row['cli_dx2_ccs'])
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid']
    if(row['cli_dx3_ccs'] !=0): 
        col="CCS" +str(row['cli_dx3_ccs']) 
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid']   
    if(row['cli_dx4_ccs'] !=0): 
        col="CCS" +str(row['cli_dx4_ccs']) 
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid'] 
    
    

In [30]:
print(diabetic_patients_binary.shape)  
print(diabetic_patients_comm_ccs.shape)

(729492, 284)
(729492, 5)


In [31]:
#filling NA's as 0
diabetic_patients_binary=diabetic_patients_binary.fillna(0) 

#Grouping by each patients
diabetic_patients_binary=diabetic_patients_binary.groupby(['new_euid']).sum()
print(diabetic_patients_binary.shape)  

(8956, 283)


In [32]:
diabetic_patients_binary.head() 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100114,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,3,0,0,3,0
10013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10014,0,0,15,30,15,1,0,3,25,0,...,0,0,0,0,0,32,10,4,5,0
100397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
## changing all values greater >1 =1
def f1(x):
    if x>=1:
        return 1
    else:
        return 0

diabetic_patients_binary=diabetic_patients_binary.applymap(f1)
diabetic_patients_binary.head()  

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100114,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
10013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10014,0,0,1,1,1,1,0,1,1,0,...,0,0,0,0,0,1,1,1,1,0
100397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Caveat: After converting to binary we are losing the information how many times a particular ccs for a particular patient was claimed

In [34]:
print(diabetic_patients_binary.shape) 

# Removing CCS49 i,e Diabetes with no complication
del diabetic_patients_binary['CCS49']  

#Removing Exam/Eval attribute.
del diabetic_patients_binary['CCS256']   

## Removing  CCS Codes which are 0 for all patients
diabetic_patients_binary=diabetic_patients_binary.loc[:,(diabetic_patients_binary!=0).any(axis=0)] 
print(diabetic_patients_binary.shape) 

(8956, 283)
(8956, 272)


# Making summary of CCS_codes for 9K diabetic patients

In [35]:
df=diabetic_patients_binary
codes_stats=pd.DataFrame(df.sum())
codes_stats.reset_index(inplace=True) 
codes_stats=codes_stats.rename(columns={0:'Total_Patients'})
codes_stats.head() 

,index,Total_Patients
0,CCS1,5
1,CCS10,3844
2,CCS100,124
3,CCS101,1258
4,CCS102,1106


In [36]:
ccs_codes.head() 

,START,LABEL,ccs_code
0,1,Tuberculosis,CCS1
1,10,Immuniz/scrn,CCS10
2,100,Acute MI,CCS100
3,101,Coron athero,CCS101
4,102,Chest pain,CCS102


In [37]:
tmp1=ccs_codes[['ccs_code','LABEL']]
codes_stats=pd.merge(tmp1,codes_stats,left_on='ccs_code',right_on='index')
del codes_stats['index'] 
codes_stats=codes_stats.sort_values(['Total_Patients'],ascending=False)
codes_stats['population_percentage']=(codes_stats['Total_Patients']*1.0/8955)*100.0
codes_stats.head() 

,ccs_code,LABEL,Total_Patients,population_percentage
217,CCS53,Hyperlipidem,6462,72.160804
270,CCS98,HTN,6098,68.096036
214,CCS50,DiabMel w/cm,4486,50.094919
1,CCS10,Immuniz/scrn,3844,42.925740
168,CCS258,Other screen,3612,40.335008


In [ ]:
#Writing summary to csv file
# codes_stats.to_csv("codes_stats.csv")

In [38]:
##Deduplicating the binary matrix
df=diabetic_patients_binary  
df_new=pd.DataFrame(df.groupby(df.columns.tolist(),as_index=False).size())  
print(df_new.shape)
df_new.reset_index(inplace=True)  
df_new=df_new.rename(columns={0:'Total_Patients'})
df_new.sort_values(['Total_Patients'],ascending=False,inplace=True)
diabetic_patients_binary_ddup=df_new
diabetic_patients_binary_ddup.head() 

(8667, 1)


,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,37
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,28
104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
4887,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,11
73,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,10


# Performing Clustering Using Cosine Similarity

In [39]:
cols=diabetic_patients_binary_ddup.columns.tolist()
diabetic_patients_binary_ddup[cols]=diabetic_patients_binary_ddup[cols].astype(float)

##converting binary matrix to IDF (i,e converting 0-1 to IDF)
for index,row in diabetic_patients_binary_ddup.iterrows():
    #print(index)
    for col in cols[:-1]:
         if(row[col]==1):
            fre=codes_stats[codes_stats['ccs_code']==col]['population_percentage']
            fre=(fre*1.0/100).iloc[0]
            #print("frequency is",fre)
            diabetic_patients_binary_ddup.loc[index][col]=np.log(1.0/fre)*1.0 
            #print("code is",col,"frequency is" ,fre,"IDF is",np.log(1.0/fre)*1.0,"Valu
diabetic_patients_binary_ddup.head() 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,37.0
39,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0,28.0
104,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.0
4887,0.0,0.845699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0,11.0
73,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0,10.0


In [41]:
print(diabetic_patients_binary_ddup.shape)
#print(clusters.shape) 

(8667, 273)


In [42]:
#checking if the conversion happens correctly
print(codes_stats[codes_stats['ccs_code']=='CCS10'])  
print(math.log(1.0/0.4292)) 

print('\n') 

print(codes_stats[codes_stats['ccs_code']=='CCS98'])  
print(math.log(1.0/0.6809))  

#Happened Correctly

  ccs_code         LABEL  Total_Patients  population_percentage
1    CCS10  Immuniz/scrn            3844               42.92574
0.845832268226


    ccs_code LABEL  Total_Patients  population_percentage
270    CCS98   HTN            6098              68.096036
0.384339826493


In [43]:
#Clustering based on Cosine Metric and Average linkage Method
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram    
from scipy.cluster.hierarchy import cophenet

checkx=diabetic_patients_binary_ddup[1:]
X=pdist(checkx.ix[:,0:len(diabetic_patients_binary_ddup.columns)-1],metric='cosine')

Z = linkage(X,method='average') 
c,cd=cophenet(Z,X)
c 

0.33388724557543031

In [44]:
##Code to make dendogram

# plt.title('Hierarchical Clustering Dendrogram (truncated)')
# plt.xlabel('Patient Group')
# plt.ylabel(' Cosine distance')
# dendrogram(
#     Z,
#     truncate_mode='level',  # show only the last p merged clusters
#     p=100,  # show only the last p merged clusters
#     show_leaf_counts=True,  # otherwise numbers in brackets are counts
#     leaf_rotation=90.,
#     leaf_font_size=12.,
#     labels=list(checkx.index),
#     show_contracted=True,  # to get a distribution impression in truncated branches
# ) 
# plt.show() 

In [45]:
#Code for clustering

from scipy.cluster.hierarchy import dendrogram, linkage , fcluster
max_d=0.9
clusters=fcluster(Z,max_d,criterion='distance')
clusters

clusters=pd.DataFrame(clusters)
clusters=clusters.rename(columns = {0:'clusters_id'})  

#concatinating binary ddup with cluster id.
checkx_new=pd.concat([checkx.reset_index(),clusters], axis=1)  
cols = checkx_new.columns.tolist() 
cols = cols[-2:] + cols[:-2] 
checkx_new=checkx_new[cols] 
checkx_new.set_index('index',inplace=True)  
community=checkx_new.groupby(['clusters_id']).Total_Patients.sum().sort_values(ascending=False)
community[:20] 

clusters_id
40    1119.0
47     936.0
25     810.0
32     610.0
21     580.0
37     502.0
39     431.0
34     406.0
26     333.0
38     266.0
28     250.0
16     246.0
44     168.0
42     161.0
33     153.0
50     139.0
27     132.0
46     121.0
36     116.0
17     109.0
Name: Total_Patients, dtype: float64

In [46]:
### code for adding Z_score and cluster statistics.

def z_score(ccs_code):
    
    p_1=cluster_stats[cluster_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/community[cluster_stats['cluster_id'][0]]
    p_1=p_1.iloc[0]
    p_2=codes_stats[codes_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/8955.0
    p_2=p_2.iloc[0]
    
    n_1 = community[cluster_stats['cluster_id'][0]]*1.0 #cluster Population
    n_2 = 8955.0  #Total Population 
  
    p =(n_1*p_1 + n_2*p_2)*1.0/(n_1 + n_2)
   
    num = (p_1-p_2)*1.0
    deno= math.sqrt(p*(1-p)*( (1.0/n_1) + (1.0/n_2) ))
    
    z=round((num*1.0/deno),2)  
    
    p_values = round((scipy.stats.norm.sf(abs(z))*2),4) #twosided P Tailed test
    
    return(z,p_values)


def clusterstatistic(cluster_id):
    #print(cluster_id)
    cluster=checkx_new[checkx_new['clusters_id']==cluster_id]
   
    cluster=cluster.loc[:,(cluster!=0).any(axis=0)]
    #cluster.head()

    ccs_codes=cluster.columns.tolist()
    ccs_codes=ccs_codes[2:]
    cluster_stats=pd.DataFrame(columns=['cluster_id','CCS_Code','Total_Patients'])
    for code in ccs_codes:
        patients=cluster[cluster[code]!=0].Total_Patients.sum() 
        #print("code is",code,"patients is",patients)
        tmp=pd.DataFrame(index=range(1,2),columns=['cluster_id','CCS_Code','Total_Patients']) 
        tmp.ix[:,0]=cluster_id
        tmp.ix[:,1]=code
        tmp.ix[:,2]=patients 
        cluster_stats=cluster_stats.append(tmp) 

    #cluster_stats.head()

    #cluster_stats=cluster_stats[1:]
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    #cluster_stats.head()

    cluster_stats['%community']=(cluster_stats['Total_Patients']*1.0/cluster['Total_Patients'].sum())*100.0
    #cluster_stats['%diabetic_population']=(cluster_stats['Total_Patients']*1.0/14845.0)*100.0
    #cluster_stats.head()

    #tmp1.head()

    cluster_stats=pd.merge(tmp1,cluster_stats,left_on='ccs_code',right_on='CCS_Code')
    #cluster_stats.head()

    del cluster_stats['CCS_Code']
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    return cluster_stats

In [122]:
community[:20]


clusters_id
40    1119.0
47     936.0
25     810.0
32     610.0
21     580.0
37     502.0
39     431.0
34     406.0
26     333.0
38     266.0
28     250.0
16     246.0
44     168.0
42     161.0
33     153.0
50     139.0
27     132.0
46     121.0
36     116.0
17     109.0
Name: Total_Patients, dtype: float64

In [49]:
import scipy.stats
cluster_stats=clusterstatistic(47)
cluster_stats['%diabetic_population']=np.nan
cluster_stats['z_score']=np.nan
cluster_stats['p_value']=np.nan
for index,row in cluster_stats.iterrows(): 
    z,p=z_score(row['ccs_code'])
    cluster_stats.loc[index,'%diabetic_population']=codes_stats[codes_stats['ccs_code']==row['ccs_code']] .population_percentage.iloc[0] 
    cluster_stats.loc[index,'z_score']=z
    cluster_stats.loc[index,'p_value']=p  

cluster_stats['sort']=cluster_stats.z_score.abs()
cluster_stats.sort_values(by='sort',ascending=False,inplace=True)
cluster_stats.drop('sort',axis=1,inplace=True)
cluster_stats  ;

In [199]:
#Code  to save all clusters to worksheets
# d=0.9
# writer = pd.ExcelWriter('../../Data_Created/Diabetic Clustering/clusters-'+str(d)+'.xlsx',engine='xlsxwriter')
# for cluster_id in community[community>80].index:
#     community_patients=community[cluster_id]
#     cluster_stats=clusterstatistic(cluster_id)
#     cluster_stats['%diabetic_population']=np.nan
#     cluster_stats['z_score']=np.nan
#     cluster_stats['p_value']=np.nan
#     for index,row in cluster_stats.iterrows(): 
#         z,p=z_score(row['ccs_code'])
#         cluster_stats.loc[index,'%diabetic_population']=codes_stats[codes_stats['ccs_code']==row['ccs_code']] .population_percentage.iloc[0] 
#         cluster_stats.loc[index,'z_score']=z
#         cluster_stats.loc[index,'p_value']=p  

#     cluster_stats['sort']=cluster_stats.z_score.abs()
#     cluster_stats.sort_values(by='sort',ascending=False,inplace=True)
#     cluster_stats.drop('sort',axis=1,inplace=True)
#     cluster_stats.to_excel(writer,sheet_name=str(int(community_patients)),index=False)
# writer.save()
      

# Performance evaluation measures on finalised clusters

In [50]:
final_clusters=[40,47,25,32,21,37,39,34,26,38,28,16] 
checkx_new.head() 

,Total_Patients,clusters_id,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
index,,,,,,,,,,,,,,,,,,,,,
39,28.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
104,19.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4887,11.0,40,0.0,0.845699,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
73,10.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
163,10.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0


In [51]:
checkx_new['Total_Patients'].sum() 

8919.0

## Mapping cluster_id back  to patient's euid & Comm NPI


In [52]:
checkx_new.shape 

(8666, 274)

In [53]:
diabetic_imp_clusters=checkx_new[checkx_new['clusters_id'].isin(final_clusters)] 

In [54]:
diabetic_imp_clusters.head() 

,Total_Patients,clusters_id,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
index,,,,,,,,,,,,,,,,,,,,,
39,28.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
104,19.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4887,11.0,40,0.0,0.845699,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
73,10.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0
163,10.0,40,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384251,0.0


In [55]:
diabetic_imp_clusters['Total_Patients'].sum() 

6489.0

### changing all values greater !0 to 1(back to binary)

In [56]:
def f1(x):
    if x!=0:
        return 1
    else:
        return 0
cols=diabetic_imp_clusters.columns.tolist()
cols=cols[2:]
diabetic_imp_clusters[cols]=diabetic_imp_clusters[cols].applymap(f1)
diabetic_imp_clusters.head() 

C:\Users\IBM_ADMIN\Anaconda2\lib\site-packages\pandas\core\frame.py:2369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Total_Patients,clusters_id,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
index,,,,,,,,,,,,,,,,,,,,,
39,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
104,19.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4887,11.0,40,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
73,10.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163,10.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [57]:
print(diabetic_imp_clusters.shape)
print(diabetic_patients_binary.shape) 

(6246, 274)
(8956, 272)


In [58]:
diabetic_patients_binary[:3] 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100114,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
10013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [60]:
#del diabetic_patients_binary['index']
diabetic_imp_clusters[:3]

,Total_Patients,clusters_id,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
index,,,,,,,,,,,,,,,,,,,,,
39,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
104,19.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4887,11.0,40,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Merging with patient_label npi to get euid-npi_comm_cluster mapping

In [61]:
diabetic_patients_binary.reset_index(inplace=True)
cols=diabetic_imp_clusters.columns.tolist()
common_cols=cols[2:] 
diabetic_euid_cluster=pd.merge(diabetic_imp_clusters,diabetic_patients_binary,on=common_cols,how='inner')

In [62]:
diabetic_euid_cluster.head() 
#so now we have mapping of euid-cluster id

,Total_Patients,clusters_id,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,new_euid
0,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,13932
1,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,173638
2,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,18670
3,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,195638
4,28.0,40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,228783


In [63]:
diabetic_euid_cluster.shape

(6489, 275)

In [64]:
#Total patients in finalised 12 cluster
len(diabetic_euid_cluster['new_euid'])

6489

In [65]:
#checking to see if clusters are mapped correctly
diabetic_euid_cluster.groupby(diabetic_euid_cluster['clusters_id']).new_euid.nunique()

clusters_id
16     246
21     580
25     810
26     333
28     250
32     610
34     406
37     502
38     266
39     431
40    1119
47     936
Name: new_euid, dtype: int64

In [66]:
#Extracting EUID-Cluster_id just
euid_cluster=diabetic_euid_cluster[['new_euid','clusters_id']]
euid_cluster[:2] 

,new_euid,clusters_id
0,13932,40
1,173638,40


In [121]:
euid_cluster[:5] 

,new_euid,clusters_id
0,13932,40
1,173638,40
2,18670,40
3,195638,40
4,228783,40


In [68]:
patient_label_npi[:2]

,new_euid,comm_npi
0,44470,1
1,498737,6


In [69]:
#Now generating euid-npi-cluster_id mapping
euid_npi_cluster=pd.merge(patient_label_npi,euid_cluster,how='inner',on='new_euid') 

In [70]:
euid_npi_cluster.shape 

(6489, 3)

In [71]:
euid_npi_cluster.head() 

,new_euid,comm_npi,clusters_id
0,44470,1,40
1,33055,1,21
2,187443,6,21
3,118528,1,16
4,395275,1,40


In [123]:
#writing above mapping to csv  
# euid_npi_cluster.to_csv("euid_npi_cluster.csv")

In [72]:
#Checking patients in each community
euid_npi_cluster.groupby(['comm_npi']).new_euid.nunique() 

comm_npi
1    2133
2    1393
3    1530
4     286
5     613
6     534
Name: new_euid, dtype: int64

# Grouping to see patients in each comm-cluster combination

In [73]:
euid_npi_cluster_grouped=pd.DataFrame(euid_npi_cluster.groupby(['comm_npi','clusters_id']).new_euid.nunique().sort_values(ascending=False))
euid_npi_cluster_grouped.reset_index(inplace=True) 
euid_npi_cluster_grouped_pivot= euid_npi_cluster_grouped.pivot(index='comm_npi',columns='clusters_id',values='new_euid')
euid_npi_cluster_grouped_pivot['Total_Patients']=euid_npi_cluster_grouped_pivot.sum(axis=1) 
euid_npi_cluster_grouped_pivot.loc['Total']=euid_npi_cluster_grouped_pivot.sum()
euid_npi_cluster_grouped_pivot 

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,Total_Patients
comm_npi,,,,,,,,,,,,,
1,77,187,257,109,85,220,144,161,76,162,390,265,2133
2,60,130,197,67,56,143,82,105,57,79,247,170,1393
3,52,147,183,84,58,124,92,127,67,85,235,276,1530
4,14,17,25,17,9,29,18,27,9,16,50,55,286
5,20,54,81,26,23,46,40,44,28,51,102,98,613
6,23,45,67,30,19,48,30,38,29,38,95,72,534
Total,246,580,810,333,250,610,406,502,266,431,1119,936,6489


In [845]:
#writing to csv
# euid_npi_cluster_grouped_pivot.to_csv("patient_number_distribution.csv")

## 1) Distribution of diabetic patients cluster within each NPI community

In [901]:
comm_clust=euid_npi_cluster_grouped_pivot
comm_clust_new=(comm_clust.ix[:6,:12].div(comm_clust['Total_Patients'],axis=0))*100
comm_clust_new=comm_clust_new.round(1) 
comm_clust_new=pd.concat([comm_clust_new,comm_clust['Total_Patients']],axis=1)  
comm_clust_new=comm_clust_new.drop(['Total'])
comm_clust_new=comm_clust_new.append(comm_clust.loc['Total'])

In [902]:
comm_clust_new

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,Total_Patients
comm_npi,,,,,,,,,,,,,
1,3.6,8.8,12.0,5.1,4.0,10.3,6.8,7.5,3.6,7.6,18.3,12.4,2133
2,4.3,9.3,14.1,4.8,4.0,10.3,5.9,7.5,4.1,5.7,17.7,12.2,1393
3,3.4,9.6,12.0,5.5,3.8,8.1,6.0,8.3,4.4,5.6,15.4,18.0,1530
4,4.9,5.9,8.7,5.9,3.1,10.1,6.3,9.4,3.1,5.6,17.5,19.2,286
5,3.3,8.8,13.2,4.2,3.8,7.5,6.5,7.2,4.6,8.3,16.6,16.0,613
6,4.3,8.4,12.5,5.6,3.6,9.0,5.6,7.1,5.4,7.1,17.8,13.5,534
Total,246.0,580.0,810.0,333.0,250.0,610.0,406.0,502.0,266.0,431.0,1119.0,936.0,6489


In [848]:
#Writing this to csv
# comm_clust_new.to_csv("cluster_within_community_patients.csv")

## 2) Distribution of NPI comm within each cluster

In [903]:
clust_comm=euid_npi_cluster_grouped_pivot
clust_comm_new=(clust_comm.ix[:6,:12]/(clust_comm.loc['Total']))*100.0
clust_comm_new=clust_comm_new.round(2)
clust_comm_new=clust_comm_new.append(clust_comm.loc['Total'])
del clust_comm_new['Total_Patients']
clust_comm_new=pd.concat([clust_comm_new,clust_comm['Total_Patients']],axis=1) 
clust_comm_new  

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,Total_Patients
comm_npi,,,,,,,,,,,,,
1,31.30,32.24,31.73,32.73,34.0,36.07,35.47,32.07,28.57,37.59,34.85,28.31,2133
2,24.39,22.41,24.32,20.12,22.4,23.44,20.20,20.92,21.43,18.33,22.07,18.16,1393
3,21.14,25.34,22.59,25.23,23.2,20.33,22.66,25.30,25.19,19.72,21.00,29.49,1530
4,5.69,2.93,3.09,5.11,3.6,4.75,4.43,5.38,3.38,3.71,4.47,5.88,286
5,8.13,9.31,10.00,7.81,9.2,7.54,9.85,8.76,10.53,11.83,9.12,10.47,613
6,9.35,7.76,8.27,9.01,7.6,7.87,7.39,7.57,10.90,8.82,8.49,7.69,534
Total,246.00,580.00,810.00,333.00,250.0,610.00,406.00,502.00,266.00,431.00,1119.00,936.00,6489


In [905]:
#writing to csv
# clust_comm_new.to_csv("community_within_cluster.csv")

### Adding pmpm_cost to euid-comm-cluster mapping

In [906]:
# reading data from pmpm_cost
type={'new_euid':'str','cost':'float','months':'float'}
pmpm_cost=pd.read_csv("../../Data/Diabetic_Patient_Clustering/pmpm_cost.csv",dtype=type)

In [907]:
pmpm_cost.head()

,new_euid,cost,months
0,0000003,193.67,12.0
1,0000004,1025.41,12.0
2,0000005,1503.28,12.0
3,0000006,4179.89,12.0
4,0000007,2020.39,12.0


In [908]:
pmpm_cost.dtypes

new_euid     object
cost        float64
months      float64
dtype: object

### Checking length of euid columns

In [909]:
pmpm_cost.groupby(pmpm_cost['new_euid'].str.len()).agg({'new_euid':np.size})

,new_euid
new_euid,
7.0,536950


### Cleaning the front 0's from euid 

In [910]:
pmpm_cost['new_euid'] = pmpm_cost['new_euid'].map(lambda x: str(x).lstrip('0'))
pmpm_cost.groupby(pmpm_cost['new_euid'].str.len()).agg({'new_euid':np.size})

,new_euid
new_euid,
1,7
2,57
3,685
4,6566
5,62406
6,467230


### Checking if new_euid is unique in pmpm_cost

In [853]:
print(pmpm_cost.shape[0])
print(len(pmpm_cost['new_euid'].unique()))  

#it is

536951
536951


### Merging with euid-comm_npi-cluster

In [911]:
euid_npi_cluster[:2] 

,new_euid,comm_npi,clusters_id
0,44470,1,40
1,33055,1,21


In [912]:
euid_npi_cluster_cost=pd.merge(euid_npi_cluster,pmpm_cost,how='inner')
print(patient_label_npi.shape)
print(euid_npi_cluster_cost.shape) 
euid_npi_cluster_cost.head() 

(8955, 2)
(6489, 5)


,new_euid,comm_npi,clusters_id,cost,months
0,44470,1,40,12223.45,12.0
1,33055,1,21,7636.60,12.0
2,187443,6,21,20686.80,12.0
3,118528,1,16,13323.18,12.0
4,395275,1,40,7369.39,12.0


### Adding average monthly cost of each patient 

In [913]:
euid_npi_cluster_cost['AvgMonthlyCost']=euid_npi_cluster_cost['cost']/euid_npi_cluster_cost['months']
euid_npi_cluster_cost.drop(['cost','months'],axis=1,inplace=True)
euid_npi_cluster_cost=euid_npi_cluster_cost.round(2)
euid_npi_cluster_cost[:2] 

,new_euid,comm_npi,clusters_id,AvgMonthlyCost
0,44470,1,40,1018.62
1,33055,1,21,636.38


### Grouping to see Avg Monthly Cost metrics across comm-clusters 

In [914]:
euid_npi_cluster_cost_grp=euid_npi_cluster_cost.groupby(['comm_npi','clusters_id']).agg({'new_euid':np.size,'AvgMonthlyCost':np.sum})
euid_npi_cluster_cost_grp.reset_index(inplace=True)
euid_npi_cluster_cost_grp[:2]  

,comm_npi,clusters_id,new_euid,AvgMonthlyCost
0,1,16,77,89675.96
1,1,21,187,185330.72


### Pivoting for Total patients across patients & communities

In [915]:
clust_comm= euid_npi_cluster_cost_grp.pivot(index='comm_npi',columns='clusters_id',values='new_euid')
clust_comm['Total_Patients']=clust_comm.sum(axis=1) 
clust_comm.loc['Total']=clust_comm.sum()
clust_comm 

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,Total_Patients
comm_npi,,,,,,,,,,,,,
1,77,187,257,109,85,220,144,161,76,162,390,265,2133
2,60,130,197,67,56,143,82,105,57,79,247,170,1393
3,52,147,183,84,58,124,92,127,67,85,235,276,1530
4,14,17,25,17,9,29,18,27,9,16,50,55,286
5,20,54,81,26,23,46,40,44,28,51,102,98,613
6,23,45,67,30,19,48,30,38,29,38,95,72,534
Total,246,580,810,333,250,610,406,502,266,431,1119,936,6489


### Pivoting for Avg Monthly cost across community-cluster

In [921]:
euid_npi_clst_cost_grp_piv= euid_npi_cluster_cost_grp.pivot(index='comm_npi',columns='clusters_id',values='AvgMonthlyCost')
comm_clst_cost=euid_npi_clst_cost_grp_piv
comm_clst_cost.loc['AvgMonthCost_clst']=comm_clst_cost.sum()
comm_clst_cost['AvgMonthCost_comm']=comm_clst_cost.sum(axis=1)
comm_clst_cost  

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,AvgMonthCost_comm
comm_npi,,,,,,,,,,,,,
1,89675.96,185330.72,343276.56,80958.35,58485.95,176468.84,184001.17,161100.74,68517.19,127829.82,267093.99,640965.45,2383704.74
2,51262.03,160063.24,244605.94,38999.81,34112.71,100083.62,145704.92,111129.87,41529.53,57003.40,105792.06,284062.03,1374349.16
3,43416.88,105004.92,198134.98,31582.22,26026.40,92178.78,129984.87,124742.81,45686.83,46830.39,101396.45,494850.77,1439836.30
4,11639.19,23307.07,28100.04,16221.03,4993.43,25485.44,104820.17,18199.33,9622.90,14507.50,70726.42,334876.34,662498.86
5,29321.22,35299.08,85259.60,12199.57,22982.70,35700.77,94835.23,53195.43,22993.66,22266.63,44450.66,162236.78,620741.33
6,19661.67,41174.60,74047.61,18799.26,14298.01,47623.26,69121.84,41640.42,24394.41,26930.15,43589.04,145285.29,566565.56
AvgMonthCost_clst,244976.95,550179.63,973424.73,198760.24,160899.20,477540.71,728468.20,510008.60,212744.52,295367.89,633048.62,2062276.66,7047695.95


In [917]:
# writing to csv
# comm_clst_cost.to_csv("community_cluster_cost_distribution.csv")

## 1). Cost Distribution of NPI communities across patient clusters

In [918]:
comm_clst_cost_new=(comm_clst_cost.ix[:6,:12].div(comm_clst_cost['AvgMonthCost_comm'],axis=0))*100
comm_clst_cost_new=comm_clst_cost_new.round(1) 
comm_clst_cost_new=pd.concat([comm_clst_cost_new,comm_clst_cost['AvgMonthCost_comm']],axis=1)  
comm_clst_cost_new=comm_clst_cost_new.drop(['AvgMonthCost_clst'])
comm_clst_cost_new=comm_clst_cost_new.append(comm_clst_cost.loc['AvgMonthCost_clst'])
comm_clst_cost_new=pd.concat([comm_clst_cost_new,clust_comm['Total_Patients']],axis=1)  
comm_clst_cost_new=comm_clst_cost_new.drop(['Total'])
comm_clst_cost_new=comm_clst_cost_new.append(clust_comm.loc['Total']) 

In [919]:
comm_clst_cost_new 

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,AvgMonthCost_comm,Total_Patients
1,3.80,7.80,14.40,3.40,2.5,7.40,7.7,6.8,2.90,5.40,11.20,26.90,2383704.74,2133.0
2,3.70,11.60,17.80,2.80,2.5,7.30,10.6,8.1,3.00,4.10,7.70,20.70,1374349.16,1393.0
3,3.00,7.30,13.80,2.20,1.8,6.40,9.0,8.7,3.20,3.30,7.00,34.40,1439836.30,1530.0
4,1.80,3.50,4.20,2.40,0.8,3.80,15.8,2.7,1.50,2.20,10.70,50.50,662498.86,286.0
5,4.70,5.70,13.70,2.00,3.7,5.80,15.3,8.6,3.70,3.60,7.20,26.10,620741.33,613.0
6,3.50,7.30,13.10,3.30,2.5,8.40,12.2,7.3,4.30,4.80,7.70,25.60,566565.56,534.0
AvgMonthCost_clst,244976.95,550179.63,973424.73,198760.24,160899.2,477540.71,728468.2,510008.6,212744.52,295367.89,633048.62,2062276.66,7047695.95,NaN
Total,246.00,580.00,810.00,333.00,250.0,610.00,406.0,502.0,266.00,431.00,1119.00,936.00,NaN,6489.0


In [920]:
#writing to csv
# comm_clst_cost_new.to_csv("cost_communities_across_clusters.csv")

## 2). Cost Distribution of clusters across NPI communities 

In [922]:
comm_clst_cost_new=(comm_clst_cost.ix[:6,:12]/(comm_clst_cost.loc['AvgMonthCost_clst']))*100.0
comm_clst_cost_new=comm_clst_cost_new.round(1) 
#comm_clst_cost_new=pd.concat([comm_clst_cost_new,comm_clst_cost['AvgMonthCost_comm']],axis=1)  
#comm_clst_cost_new=comm_clst_cost_new.drop(['AvgMonthCost_clst']) 
comm_clst_cost_new=comm_clst_cost_new.append(comm_clst_cost.loc['AvgMonthCost_clst'])
del comm_clst_cost_new['AvgMonthCost_comm']
comm_clst_cost_new=pd.concat([comm_clst_cost_new,comm_clst_cost['AvgMonthCost_comm']],axis=1)  
comm_clst_cost_new=pd.concat([comm_clst_cost_new,clust_comm['Total_Patients']],axis=1)  
comm_clst_cost_new=comm_clst_cost_new.drop(['Total']) 
comm_clst_cost_new=comm_clst_cost_new.append(clust_comm.loc['Total'])  

In [923]:
# Writing to csv
#comm_clst_cost_new

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,AvgMonthCost_comm,Total_Patients
1,36.60,33.70,35.30,40.70,36.3,37.00,25.3,31.6,32.20,43.30,42.20,31.10,2383704.74,2133.0
2,20.90,29.10,25.10,19.60,21.2,21.00,20.0,21.8,19.50,19.30,16.70,13.80,1374349.16,1393.0
3,17.70,19.10,20.40,15.90,16.2,19.30,17.8,24.5,21.50,15.90,16.00,24.00,1439836.30,1530.0
4,4.80,4.20,2.90,8.20,3.1,5.30,14.4,3.6,4.50,4.90,11.20,16.20,662498.86,286.0
5,12.00,6.40,8.80,6.10,14.3,7.50,13.0,10.4,10.80,7.50,7.00,7.90,620741.33,613.0
6,8.00,7.50,7.60,9.50,8.9,10.00,9.5,8.2,11.50,9.10,6.90,7.00,566565.56,534.0
AvgMonthCost_clst,244976.95,550179.63,973424.73,198760.24,160899.2,477540.71,728468.2,510008.6,212744.52,295367.89,633048.62,2062276.66,7047695.95,NaN
Total,246.00,580.00,810.00,333.00,250.0,610.00,406.0,502.0,266.00,431.00,1119.00,936.00,NaN,6489.0


In [924]:
comm_clst_cost_new.to_csv("cost_cluster_across_communities.csv")

# Adding Gender ,Pcal,Average Age information to community cluster

In [939]:
#Reading pmpm to get pcal score
cols=['new_euid','pcal']
type= {'new_euid':'str','pcal':'str'}
pmpm=pd.read_csv("../../Data/pmpm.csv",usecols=cols,dtype=type)

In [940]:
pmpm.head()

,new_euid,pcal
0,0718157,1.160
1,0718158,1.473
2,0718159,0.861
3,0718161,2.592
4,0718162,1.777


In [941]:
#checking new_euid length
pmpm.groupby(pmpm['new_euid'].str.len()).agg({'new_euid':np.size})

,new_euid
new_euid,
7.0,460451


In [942]:
pmpm['new_euid']=pmpm['new_euid'].map(lambda x:str(x).lstrip('0'))

In [943]:
pmpm.groupby(pmpm['new_euid'].str.len()).agg({'new_euid':np.size})

,new_euid
new_euid,
1,7
2,52
3,624
4,5894
5,55460
6,398415


In [944]:
##Merging with euid_npi_cluster
euid_npi_cluster[:2]

,new_euid,comm_npi,clusters_id
0,44470,1,40
1,33055,1,21


In [945]:
euid_npi_cluster_pcal=pd.merge(euid_npi_cluster,pmpm,how='inner')

In [947]:
euid_npi_cluster_pcal[:2]

,new_euid,comm_npi,clusters_id,pcal
0,44470,1,40,2.585
1,33055,1,21,2.454


In [948]:
euid_npi_cluster_pcal.isnull().sum() 
#perfect (all patients mapped to pcal)

new_euid       0
comm_npi       0
clusters_id    0
pcal           0
dtype: int64

In [950]:
#Reading Diabetes all claims to get gender,age information
cols=['new_euid','GENDER','yob']
type={'new_euid':'str','GENDER':'str'}
diab_all_claims=pd.read_csv("../../Data/Diabetic_Patient_Clustering/diabetes_all_claims.csv",usecols=cols,dtype=type)

In [961]:
diab_all_claims[:2]

,new_euid,GENDER,Age
0,0604865,F,52
1,0604865,F,52


In [953]:
diab_all_claims.dtypes

new_euid    object
GENDER      object
yob          int64
dtype: object

In [964]:
#stripping front 0's 
diab_all_claims['new_euid']=diab_all_claims['new_euid'].map(lambda x:str(x).lstrip('0'))

In [954]:
#Adding age
diab_all_claims['Age']=2014-diab_all_claims['yob']+1

In [958]:
diab_all_claims.head()
diab_all_claims=diab_all_claims[['new_euid','GENDER','Age']]

In [965]:
### Merging with euid_npi_cluster_pcal
euid_npi_cluster_pcal_demo=pd.merge(euid_npi_cluster_pcal,diab_all_claims) 

In [966]:
euid_npi_cluster_pcal_demo.head()

,new_euid,comm_npi,clusters_id,pcal,GENDER,Age
0,44470,1,40,2.585,F,62
1,44470,1,40,2.585,F,62
2,44470,1,40,2.585,F,62
3,44470,1,40,2.585,F,62
4,44470,1,40,2.585,F,62


In [968]:
euid_npi_cluster_pcal_demo.isnull().sum()
##Perfect

new_euid       0
comm_npi       0
clusters_id    0
pcal           0
GENDER         0
Age            0
dtype: int64

In [971]:
#dropping duplicates
df=euid_npi_cluster_pcal_demo
df=df.drop_duplicates()

In [972]:
df.shape

(6489, 6)

In [973]:
df.isnull().sum()

new_euid       0
comm_npi       0
clusters_id    0
pcal           0
GENDER         0
Age            0
dtype: int64

In [974]:
df.head()

,new_euid,comm_npi,clusters_id,pcal,GENDER,Age
0,44470,1,40,2.585,F,62
81,33055,1,21,2.454,F,61
193,187443,6,21,6.621,F,62
376,118528,1,16,0.983,F,37
462,395275,1,40,1.917,M,52


### Pivoting for gender distribution across community-cluster

In [975]:
##Agregating gender count across comm-cluster 
df_female=df.groupby(['comm_npi','clusters_id','GENDER']).size()
df_female.head()


comm_npi  clusters_id  GENDER
1         16           F          38
                       M          39
          21           F          38
                       M         149
          25           F         119
dtype: int64

In [978]:
df_female=pd.DataFrame(df_female)
df_female.reset_index(inplace=True)
df_female.head()

,comm_npi,clusters_id,GENDER,0
0,1,16,F,38
1,1,16,M,39
2,1,21,F,38
3,1,21,M,149
4,1,25,F,119


In [979]:
##Dropping male  
df_female=df_female[df_female['GENDER']=='F']
df_female=df_female.rename(columns = {0:'Total_Female'})
df_female.head()

,comm_npi,clusters_id,GENDER,Total_Female
0,1,16,F,38
2,1,21,F,38
4,1,25,F,119
6,1,26,F,52
8,1,28,F,46


In [980]:
df_female_pivot= df_female.pivot(index='comm_npi',columns='clusters_id',values='Total_Female')


In [981]:
df_female_pivot.head()

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47
comm_npi,,,,,,,,,,,,
1,38,38,119,52,46,208,47,61,38,53,154,77
2,27,22,88,28,27,141,35,41,27,33,88,50
3,21,19,86,41,21,119,31,56,43,34,83,81
4,10,2,14,12,5,29,5,11,7,11,25,14
5,13,8,43,10,11,46,13,19,13,24,41,35


In [986]:
## Rearranging columns as per decreasing avg monthly cost per patient.
cols=[47,34,37,25,16,21,38,32,39,28,26,40]
df_female_pivot=df_female_pivot[cols]
df_female_pivot.head()

clusters_id,47,34,37,25,16,21,38,32,39,28,26,40
comm_npi,,,,,,,,,,,,
1,77,47,61,119,38,38,38,208,53,46,52,154
2,50,35,41,88,27,22,27,141,33,27,28,88
3,81,31,56,86,21,19,43,119,34,21,41,83
4,14,5,11,14,10,2,7,29,11,5,12,25
5,35,13,19,43,13,8,13,46,24,11,10,41


In [987]:
df_female_pivot.to_csv("df_female_pivot.csv")

### Pivoting for age,pcal distribution across community-cluster

In [988]:
df.head()

,new_euid,comm_npi,clusters_id,pcal,GENDER,Age
0,44470,1,40,2.585,F,62
81,33055,1,21,2.454,F,61
193,187443,6,21,6.621,F,62
376,118528,1,16,0.983,F,37
462,395275,1,40,1.917,M,52


In [994]:
df.dtypes

new_euid       object
comm_npi        int64
clusters_id     int32
pcal           object
GENDER         object
Age             int64
dtype: object

In [995]:
df['pcal']=df['pcal'].astype(float)

C:\Users\IBM_ADMIN\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [996]:
df_age_pcal=df.groupby(['comm_npi','clusters_id']).agg({'Age':np.sum,'pcal':np.sum})

In [998]:
df_age_pcal.reset_index(inplace=True)
df_age_pcal.head()

,comm_npi,clusters_id,pcal,Age
0,1,16,346.874,4187
1,1,21,701.751,11083
2,1,25,1104.351,14513
3,1,26,333.585,5865
4,1,28,263.763,4623


In [999]:
df_pcal_pivot= df_age_pcal.pivot(index='comm_npi',columns='clusters_id',values='pcal')
df_age_pivot= df_age_pcal.pivot(index='comm_npi',columns='clusters_id',values='Age')
## Rearranging columns as per decreasing avg monthly cost per patient.
cols=[47,34,37,25,16,21,38,32,39,28,26,40]
df_pcal_pivot=df_pcal_pivot[cols] 
df_age_pivot=df_age_pivot[cols]

In [1000]:
df_pcal_pivot[:2]

clusters_id,47,34,37,25,16,21,38,32,39,28,26,40
comm_npi,,,,,,,,,,,,
1,1468.027,619.636,521.682,1104.351,346.874,701.751,247.857,741.817,520.214,263.763,333.585,904.390
2,825.386,367.904,353.614,842.591,227.047,513.679,172.493,448.693,235.944,149.437,195.243,503.137


In [1001]:
df_age_pivot[:2]

clusters_id,47,34,37,25,16,21,38,32,39,28,26,40
comm_npi,,,,,,,,,,,,
1,16080,8305,8884,14513,4187,11083,4077,12465,9474,4623,5865,21380
2,10282,4801,6080,11363,3293,7703,3166,7905,4550,3099,3694,13746


In [1002]:
##Rounding to 2 digit
df_pcal_pivot=df_pcal_pivot.round(1)

In [1003]:
#Writing to csv files
# df_pcal_pivot.to_csv("df_pcal_pivot.csv")
# df_age_pivot.to_csv("df_age_pivot.csv")

# Defining NPI Communities

## Subtask 1: NPI-Community : speciality

### Mapping NPI  to NPI-Speciality

In [1092]:
#Reading diabetic_all_claims only with necessary songs. 
cols=['svc_npi','svc_prov_tax_id','svc_specialty']
types={'svc_npi':'str','svc_prov_tax_id':'str','svc_specialty':'str'}
diabetic_all_claims=pd.read_csv("../../Data/Diabetic_Patient_Clustering/diabetes_all_claims.csv",usecols=cols,dtype=types)

In [1093]:
diabetic_all_claims.head() 

,svc_npi,svc_prov_tax_id,svc_specialty
0,<NA>,NaN,NOT APPLICABLE
1,<NA>,NaN,NOT APPLICABLE
2,<NA>,NaN,NOT APPLICABLE
3,<NA>,NaN,NOT APPLICABLE
4,<NA>,NaN,NOT APPLICABLE


In [1094]:
diabetic_all_claims.dtypes

svc_npi            object
svc_prov_tax_id    object
svc_specialty      object
dtype: object

In [1095]:
diabetic_all_claims.isnull().sum()

svc_npi               216
svc_prov_tax_id    292990
svc_specialty           0
dtype: int64

In [1096]:
### Removing all observation where svc_npi='<NA>'
diabetic_all_claims=diabetic_all_claims[diabetic_all_claims['svc_npi']!='<NA>']

In [1097]:
### Removing null observation
diabetic_all_claims=diabetic_all_claims[diabetic_all_claims['svc_npi'].notnull()]
diabetic_all_claims.isnull().sum()


svc_npi             0
svc_prov_tax_id    13
svc_specialty       0
dtype: int64

In [1098]:
diabetic_all_claims.shape

(496904, 3)

In [1099]:
diabetic_all_claims.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496904 entries, 48 to 790085
Data columns (total 3 columns):
svc_npi            496904 non-null object
svc_prov_tax_id    496891 non-null object
svc_specialty      496904 non-null object
dtypes: object(3)
memory usage: 15.2+ MB


In [1100]:
###Dropping duplicates 
diabetic_all_claims=diabetic_all_claims.drop_duplicates() 
diabetic_all_claims.shape

(6240, 3)

In [1047]:
## Unique provider ID
len(diabetic_all_claims['svc_npi'].unique())

5753

In [1101]:
#Taking most frequent speciality of svc_npi 
from scipy import stats
npi_speciality=diabetic_all_claims[['svc_npi','svc_specialty']].groupby(['svc_npi']).agg(lambda x:stats.mode(x['svc_specialty'])[0])
npi_speciality.reset_index(inplace=True)

In [1056]:
npi_speciality.head()

,svc_npi,svc_specialty
0,1003047788,Family Medicine
1,1003054263,Obstetrics and Gynecology
2,1003065434,Physical Therapy
3,1003070749,Internal Medicine (Internal Medicine)
4,1003071341,Anesthesiology


In [1061]:
npi_speciality.shape

(5753, 2)

### Mapping NPI_speciality to NPI-Community_Label

In [1057]:
#readinf Label from nodeList
nodeList=pd.read_csv("../../Data/Diabetic_Patient_Clustering/nodeList(Cut).csv")

In [1058]:
nodeList.head()

,Id,Label
0,1043447758,1
1,1225039787,1
2,1255525358,1
3,1356433858,1
4,1538167697,1


In [1063]:
nodeList.dtypes

Id       int64
Label    int64
dtype: object

In [1064]:
npi_speciality.dtypes

svc_npi          object
svc_specialty    object
dtype: object

In [1066]:
##Changing NodeList Id to string
nodeList['Id']=nodeList['Id'].astype('str')

In [1067]:
#merging to get npi-npi_community-svc_specialty mapping
comm_npi_speciality=pd.merge(nodeList,npi_speciality,left_on="Id",right_on="svc_npi",how="inner")

In [1069]:
### All comm_npi_speciality mapped
comm_npi_speciality.shape

(2080, 4)

In [1070]:
comm_npi_speciality.head()

,Id,Label,svc_npi,svc_specialty
0,1043447758,1,1043447758,Family Medicine
1,1225039787,1,1225039787,Pathology Anatomic
2,1255525358,1,1255525358,Emergency Medicine
3,1356433858,1,1356433858,Internal Medicine (Internal Medicine)
4,1538167697,1,1538167697,Cardiovascular Diseases


In [1071]:
del comm_npi_speciality['Id']

In [1072]:
comm_npi_speciality.head()

,Label,svc_npi,svc_specialty
0,1,1043447758,Family Medicine
1,1,1225039787,Pathology Anatomic
2,1,1255525358,Emergency Medicine
3,1,1356433858,Internal Medicine (Internal Medicine)
4,1,1538167697,Cardiovascular Diseases


In [1075]:
#grouping to check major speciality of each label
comm_speciality_summary=comm_npi_speciality.groupby(['Label','svc_specialty']).agg({'svc_npi':np.size})

In [1076]:
comm_speciality_summary.reset_index(inplace=True)


In [1077]:
comm_speciality_summary.head()

,Label,svc_specialty,svc_npi
0,1,Anesthesiology,27
1,1,Audiology,5
2,1,Cardiac Electrophysiology,2
3,1,Cardiothoracic Surgery,1
4,1,Cardiovascular Diseases,37


In [1081]:
comm_speciality_summary=comm_speciality_summary.sort_values(['Label','svc_npi'],ascending=[False,False])

In [1082]:
#writing to_csv
# comm_speciality_summary.to_csv("comm_speciality_summary.csv")

## Subtask2: NPI-Community:Major Hospital

In [1083]:
#Reading hospital name from tin_name file provided in Data Dictionary (i,e tax_id:tin name mapping)
tin_name=pd.read_csv("../../Data/Diabetic_Patient_Clustering/tin_name.csv")

In [1084]:
tin_name.head()

,START,LABEL
0,**OTHER**,NaN
1,-22367848,"Pastore, Dominic"
2,-27134308,National Surgery Center
3,-38135821,Providence Park Hospital
4,-52212832,"Smakja-Gordon, Michelle"


In [1085]:
tin_name.shape

(59442, 2)

In [1086]:
tin_name.dtypes

START    object
LABEL    object
dtype: object

In [1102]:
diabetic_all_claims.head()

,svc_npi,svc_prov_tax_id,svc_specialty
48,1205820727,141834454,Cardiovascular Diseases
51,1326268491,200911087,Emergency Medicine
52,1043267727,141338544,UNKNOWN
61,1649442070,141660131,Diagnostic Radiology
62,1679531289,141660131,Obstetrics and Gynecology


In [1103]:
npi_speciality_tin_name=pd.merge(diabetic_all_claims,tin_name,left_on='svc_prov_tax_id',right_on='START')

In [1104]:
npi_speciality_tin_name.head()

,svc_npi,svc_prov_tax_id,svc_specialty,START,LABEL
0,1205820727,141834454,Cardiovascular Diseases,141834454,Capital Cardiology Associates PC
1,1891946703,141834454,Cardiovascular Diseases,141834454,Capital Cardiology Associates PC
2,1174518989,141834454,Cardiovascular Diseases,141834454,Capital Cardiology Associates PC
3,1841284361,141834454,Cardiovascular Diseases,141834454,Capital Cardiology Associates PC
4,1568457943,141834454,Cardiovascular Diseases,141834454,Capital Cardiology Associates PC


In [1107]:
npi_speciality_tin_name=npi_speciality_tin_name.drop(['svc_prov_tax_id','svc_specialty','START'],axis=1)

In [1108]:
npi_speciality_tin_name.head()

,svc_npi,LABEL
0,1205820727,Capital Cardiology Associates PC
1,1891946703,Capital Cardiology Associates PC
2,1174518989,Capital Cardiology Associates PC
3,1841284361,Capital Cardiology Associates PC
4,1568457943,Capital Cardiology Associates PC


In [1115]:
pd.DataFrame(npi_speciality_tin_name.groupby(['svc_npi']).LABEL.nunique());

In [1111]:
### Merging with community label 
comm_npi_speciality_tin_name=pd.merge(nodeList,npi_speciality_tin_name,left_on="Id",right_on="svc_npi",how="inner")

In [1112]:
comm_npi_speciality_tin_name.head()

,Id,Label,svc_npi,LABEL
0,1043447758,1,1043447758,Manning Emergency Medical
1,1225039787,1,1225039787,Maplewood Pathology PC
2,1255525358,1,1255525358,Manning Emergency Medical
3,1356433858,1,1356433858,Albany Memorial Hospital
4,1538167697,1,1538167697,St Peters Health Partners Medical Associates PC


In [1116]:
len(comm_npi_speciality_tin_name['svc_npi'].unique())

2068

In [1117]:
comm_npi_speciality_tin_name.shape

(2382, 4)

In [1118]:
comm_speciality_tin_name_summary=comm_npi_speciality_tin_name.groupby(['Label','LABEL']).agg({'svc_npi':np.size})

In [1119]:
comm_speciality_tin_name_summary.head()

svc_npi
Label LABEL                                         
1     AGAPE - A Woman's Medical Place, PLLC        1
      Adirondack Foot Ankle Center                 1
      Adult Medicine PC                            1
      Akbari, Ghulam A.                            1
      Alan Bloomberg MD PC                         2

In [1122]:
comm_speciality_tin_name_summary.reset_index(inplace=True)
comm_speciality_tin_name_summary=comm_speciality_tin_name_summary.sort_values(['Label','svc_npi'],ascending=[True,False])

In [1123]:
comm_speciality_tin_name_summary.head()

,Label,LABEL,svc_npi
146,1,St Peters Health Partners Medical Associates PC,125
145,1,St Peter's Hospital,47
11,1,Albany Medical College,36
19,1,Albany-Capital Region Gastroenterology Consult...,25
46,1,"Community Care Physicians, PC",25


In [1124]:
# comm_speciality_tin_name_summary.to_csv("comm_tin_name_summary.csv")

In [1125]:
comm_speciality_summary.head()

,Label,svc_specialty,svc_npi
247,6,Family Medicine,27
250,6,Internal Medicine (Internal Medicine),20
260,6,Physical Therapy,13
255,6,Orthopaedics,9
243,6,Chiropractor,4


In [1131]:
speciality_summary=comm_speciality_summary.groupby(['svc_specialty']).agg({'svc_npi':np.sum})

In [1133]:
speciality_summary=speciality_summary.sort_values('svc_npi',ascending=False)

In [1134]:
# speciality_summary.to_csv("speciality_summary.csv")

# Adding PQI measurement euid-npi-cluster

In [74]:
euid_npi_cluster.head()

,new_euid,comm_npi,clusters_id
0,44470,1,40
1,33055,1,21
2,187443,6,21
3,118528,1,16
4,395275,1,40


In [75]:
euid_pqi=pd.read_csv("../../data/PQI/euid_pqi.csv",dtype={'new_euid':'str','CLAIM_ID':'str'})

In [76]:
euid_pqi.head()

,new_euid,CLAIM_ID,TAPQ01,TAPQ02,TAPQ03,TAPQ05,TAPQ07,TAPQ08,TAPQ10,TAPQ11,TAPQ12,TAPQ13,TAPQ14,TAPQ15,TAPQ16,TAPQ90,TAPQ91,TAPQ92
0,32,1419100NVC00,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1,32,141770BHLT00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,32,141810CJSK00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32,141120013300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,32,141810CBHM00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
euid_pqi.shape


(383374, 18)

In [78]:
len(euid_pqi['new_euid'].unique())

42643

In [79]:
euid_pqi.dtypes

new_euid    object
CLAIM_ID    object
TAPQ01       int64
TAPQ02       int64
TAPQ03       int64
TAPQ05       int64
TAPQ07       int64
TAPQ08       int64
TAPQ10       int64
TAPQ11       int64
TAPQ12       int64
TAPQ13       int64
TAPQ14       int64
TAPQ15       int64
TAPQ16       int64
TAPQ90       int64
TAPQ91       int64
TAPQ92       int64
dtype: object

In [80]:
# Merging euid_npi_cluster information with pqi
euid_npi_cluster_pqi=pd.merge(euid_npi_cluster,euid_pqi,on="new_euid",how="left")

In [82]:
euid_npi_cluster_pqi.head()

,new_euid,comm_npi,clusters_id,CLAIM_ID,TAPQ01,TAPQ02,TAPQ03,TAPQ05,TAPQ07,TAPQ08,TAPQ10,TAPQ11,TAPQ12,TAPQ13,TAPQ14,TAPQ15,TAPQ16,TAPQ90,TAPQ91,TAPQ92
0,44470,1,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33055,1,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187443,6,21,133470BKHP00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,187443,6,21,133470BJRS00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,187443,6,21,1333000G0Q00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
len(euid_npi_cluster_pqi['new_euid'].unique())

6489

In [90]:
len(euid_npi_cluster_pqi[euid_npi_cluster_pqi['CLAIM_ID'].isnull()].new_euid.unique())

5764

In [91]:
6489-5764
#So only 725 patients have PQI 

725

In [92]:
euid_npi_cluster_pqi.shape

(15542, 20)

In [94]:
euid_npi_cluster_pqi=euid_npi_cluster_pqi.drop_duplicates()

In [95]:
euid_npi_cluster_pqi.shape

(15542, 20)

### Distribution of  725 patients across community-cluster

In [96]:
euid_npi_cluster_pqi_725=euid_npi_cluster_pqi[euid_npi_cluster_pqi['CLAIM_ID'].notnull()]
euid_npi_cluster_pqi_725.head()

,new_euid,comm_npi,clusters_id,CLAIM_ID,TAPQ01,TAPQ02,TAPQ03,TAPQ05,TAPQ07,TAPQ08,TAPQ10,TAPQ11,TAPQ12,TAPQ13,TAPQ14,TAPQ15,TAPQ16,TAPQ90,TAPQ91,TAPQ92
2,187443,6,21,133470BKHP00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,187443,6,21,133470BJRS00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,187443,6,21,1333000G0Q00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,187443,6,21,133180CCRV01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
6,187443,6,21,1331200TRS00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
euid_npi_cluster_pqi_725_grp=euid_npi_cluster_pqi_725.groupby(['comm_npi','clusters_id']).agg({'new_euid':pd.Series.nunique})
euid_npi_cluster_pqi_725_grp.reset_index(inplace=True)
euid_npi_cluster_pqi_725_grp[:2]  

,comm_npi,clusters_id,new_euid
0,1,16,11
1,1,21,10


In [118]:
euid_npi_cluster_pqi_725_claim_grp=euid_npi_cluster_pqi_725.groupby(['comm_npi','clusters_id']).agg({'CLAIM_ID':pd.Series.nunique})
euid_npi_cluster_pqi_725_claim_grp.reset_index(inplace=True)
euid_npi_cluster_pqi_725_claim_grp[:2]  

,comm_npi,clusters_id,CLAIM_ID
0,1,16,102
1,1,21,127


In [120]:
clust_comm_claim= euid_npi_cluster_pqi_725_claim_grp.pivot(index='comm_npi',columns='clusters_id',values='CLAIM_ID')
clust_comm_claim['Total_Claims']=clust_comm_claim.sum(axis=1) 
clust_comm_claim.loc['Total_Claims']=clust_comm_claim.sum()
clust_comm_claim.to_csv("clust_comm_claim.csv")

In [98]:
clust_comm= euid_npi_cluster_pqi_725_grp.pivot(index='comm_npi',columns='clusters_id',values='new_euid')
clust_comm['Total_Patients']=clust_comm.sum(axis=1) 
clust_comm.loc['Total']=clust_comm.sum()
clust_comm 

clusters_id,16,21,25,26,28,32,34,37,38,39,40,47,Total_Patients
comm_npi,,,,,,,,,,,,,
1,11.0,10.0,47.0,6.0,2.0,12.0,21.0,12.0,5.0,4.0,17.0,56.0,203.0
2,5.0,15.0,42.0,5.0,2.0,13.0,9.0,15.0,6.0,10.0,11.0,29.0,162.0
3,5.0,11.0,32.0,2.0,1.0,8.0,15.0,15.0,5.0,5.0,3.0,48.0,150.0
4,2.0,4.0,6.0,1.0,2.0,2.0,8.0,1.0,2.0,2.0,4.0,14.0,48.0
5,4.0,12.0,16.0,1.0,NaN,11.0,10.0,9.0,3.0,7.0,8.0,27.0,108.0
6,5.0,3.0,10.0,NaN,NaN,5.0,7.0,4.0,1.0,1.0,2.0,16.0,54.0
Total,32.0,55.0,153.0,15.0,7.0,51.0,70.0,56.0,22.0,29.0,45.0,190.0,725.0


In [99]:
# clust_comm.to_csv("725_cluster_community_distribution.csv")

### Analyzing PQI 1

In [100]:
euid_npi_cluster_pqi_725.head()

,new_euid,comm_npi,clusters_id,CLAIM_ID,TAPQ01,TAPQ02,TAPQ03,TAPQ05,TAPQ07,TAPQ08,TAPQ10,TAPQ11,TAPQ12,TAPQ13,TAPQ14,TAPQ15,TAPQ16,TAPQ90,TAPQ91,TAPQ92
2,187443,6,21,133470BKHP00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,187443,6,21,133470BJRS00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,187443,6,21,1333000G0Q00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,187443,6,21,133180CCRV01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
6,187443,6,21,1331200TRS00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
euid_npi_cluster_pqi_725_PQI=euid_npi_cluster_pqi_725[euid_npi_cluster_pqi_725['TAPQ08']!=0.0].groupby(['comm_npi','clusters_id']).agg({'CLAIM_ID':pd.Series.nunique,'new_euid':pd.Series.nunique})
euid_npi_cluster_pqi_725_PQI.reset_index(inplace=True) 

##Pivoting for claim
clust_comm_claim= euid_npi_cluster_pqi_725_PQI.pivot(index='comm_npi',columns='clusters_id',values='CLAIM_ID')
clust_comm_claim['Total_claims']=clust_comm_claim.sum(axis=1) 
clust_comm_claim.loc['Total_claims']=clust_comm_claim.sum()
clust_comm_claim.to_csv("claim_pqi8.csv")

 
##Pivoting for new_euid 
clust_comm_euid= euid_npi_cluster_pqi_725_PQI.pivot(index='comm_npi',columns='clusters_id',values='new_euid')
clust_comm_euid['Total_patients_with_claims']=clust_comm_euid.sum(axis=1) 
clust_comm_euid.loc['Total_patients_with_claims']=clust_comm_euid.sum()
clust_comm_euid.to_csv("euid_pqi8.csv") 